In [1]:
# do a hacky import since brainaudio requires python 3.12 and the .wfst env requires an older
# python version 
from wfst_utils import build_lm_decoder, lm_decode, arrange_logits, _cer_and_wer
import sys
import numpy as np
import pandas as pd

In [2]:
load_lm = True

In [3]:
logits_b2t_24 = pd.read_pickle("/data2/brain2text/b2t_24/tm_transformer_b2t_24+25_large_wide/logits_val.pkl")
logits_b2t_25 = pd.read_pickle("/data2/brain2text/b2t_25/tm_transformer_b2t_24+25_large_wide/logits_val.pkl")

In [5]:
lmDir = "/data2/brain2text/languageModel"
acoustic_scale = 0.8
nbest = 1
blank_penalty = np.log(2)
return_n_best = False
rescore = False
nbest = 1

if load_lm and 'ngramDecoder' not in globals(): 
    
    ngramDecoder = build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I1013 16:39:51.782960 600832 brain_speech_decoder.h:52] Reading fst /data2/brain2text/languageModel/TLG.fst
I1013 16:42:41.867743 600832 brain_speech_decoder.h:81] Reading symbol table /data2/brain2text/languageModel/words.txt


In [6]:
decoded_text_outputs = []

for logits in logits_b2t_24:
    
    rearranged_logits = arrange_logits(logits)
    
    decoded_text = lm_decode(
                    ngramDecoder,
                    rearranged_logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
    
    decoded_text_outputs.append(decoded_text.strip())

In [14]:
transcripts_24 = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val.pkl")
transcripts_24[0]

'theocracy reconsidered'

In [16]:
_cer_and_wer(decoded_text_outputs, transcripts_24)

(np.float64(0.12274668790270035), np.float64(0.18967084924531732))